# Clean and Save Cleaned Data

In [1]:
import pyspark
from pyspark.sql.types import * 

In [2]:
# sc = spark.sparkContext
spark = SparkSession(sc)

In [3]:
# devicestatus.txt - upload raw data files to S3

In [4]:
filepath = "s3a://final-kmeans/raw/devicestatus.txt"

In [5]:
# Load a text file and convert each line to a Row.
lines = sc.textFile(filepath)
lines.persist()

# Split each line by the 19th character (delimiter can be "," or "|", but always in same location)
parts = lines.map(lambda l: l.split(l[19]))

# Clean up data, remove surrounding white spaces and incomplete entries
devicestatus = parts.map(lambda p: [i.strip() for i in p])
ds = devicestatus.filter(lambda x: len(x) == 14)

In [6]:
# View an entry
ds.take(1)

[[u'2014-03-15:10:10:20',
  u'Sorrento F41L',
  u'8cc3b47e-bd01-4482-b500-28f2342679af',
  u'7',
  u'24',
  u'39',
  u'enabled',
  u'disabled',
  u'connected',
  u'55',
  u'67',
  u'12',
  u'33.6894754264',
  u'-117.543308253']]

In [7]:
# Create a dataframe and persist to prevent calling to the S3 bucket for every operation
df = spark.createDataFrame(ds)
df.persist()

DataFrame[_1: string, _2: string, _3: string, _4: string, _5: string, _6: string, _7: string, _8: string, _9: string, _10: string, _11: string, _12: string, _13: string, _14: string]

In [8]:
# View dataframe
df.show(1)

+-------------------+-------------+--------------------+---+---+---+-------+--------+---------+---+---+---+-------------+--------------+
|                 _1|           _2|                  _3| _4| _5| _6|     _7|      _8|       _9|_10|_11|_12|          _13|           _14|
+-------------------+-------------+--------------------+---+---+---+-------+--------+---------+---+---+---+-------------+--------------+
|2014-03-15:10:10:20|Sorrento F41L|8cc3b47e-bd01-448...|  7| 24| 39|enabled|disabled|connected| 55| 67| 12|33.6894754264|-117.543308253|
+-------------------+-------------+--------------------+---+---+---+-------+--------+---------+---+---+---+-------------+--------------+
only showing top 1 row



In [9]:
# Select relevant columns and impose appropriate names and datatypes
from pyspark.sql.functions import col, to_timestamp

device_df = df.selectExpr('cast(_13 as float) as latitude', 'cast(_14 as float) as longitude',
                          '_1 as date', '_2 as device','_4 as model')

device_df = device_df.withColumn('date',to_timestamp(col('date'), "yyyy-MM-dd:HH:mm:ss"))

In [10]:
# View new dataframe
device_df.show(2)

+---------+-----------+-------------------+-------------+-----+
| latitude|  longitude|               date|       device|model|
+---------+-----------+-------------------+-------------+-----+
|33.689476|-117.543304|2014-03-15 10:10:20|Sorrento F41L|    7|
| 37.43211| -121.48503|2014-03-15 10:10:20|   MeeToo 1.0|    0|
+---------+-----------+-------------------+-------------+-----+
only showing top 2 rows



In [11]:
# View dataframe schema to confirm proper datatypes
device_df.schema

StructType(List(StructField(latitude,FloatType,true),StructField(longitude,FloatType,true),StructField(date,TimestampType,true),StructField(device,StringType,true),StructField(model,StringType,true)))

In [12]:
device_df.count()

100000

In [13]:
# Remove datapoints with likely false locations and count to verify items have been removed
device_df = device_df.filter((device_df.latitude != 0.0) & (device_df.longitude != 0.0))
device_df.count()

94039

In [14]:
import pyspark.sql.functions as f

In [15]:
# Split device column into separate manufacturer and model columns
mfr = f.split(device_df.device, ' ').getItem(0).alias('manufacturer')
model_name = f.split(device_df.device, ' ').getItem(1).alias('model_name')

device_df = device_df.select('latitude','longitude', 'date', mfr, model_name, 'model')
device_df.show(2)

+---------+-----------+-------------------+------------+----------+-----+
| latitude|  longitude|               date|manufacturer|model_name|model|
+---------+-----------+-------------------+------------+----------+-----+
|33.689476|-117.543304|2014-03-15 10:10:20|    Sorrento|      F41L|    7|
| 37.43211| -121.48503|2014-03-15 10:10:20|      MeeToo|       1.0|    0|
+---------+-----------+-------------------+------------+----------+-----+
only showing top 2 rows



In [17]:
# Save dataframe to S3 bucket as a csv file
outpath = 's3a://final-kmeans/clean'
# Do not write header?
device_df.coalesce(1).write.csv(outpath+'/mobilenet.csv')